In [1]:
import openai
from openai import OpenAI

In [2]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)





In [3]:

def react_agent(user_input, conversation_history, task_plan):
    """
    一个适配中文的 ReAct Agent，用于多轮对话中处理用户输入，思考、行动、观察并规划任务。

    参数：
        user_input (str): 用户输入的问题或陈述。
        conversation_history (list): 对话历史，格式为 [(thought, action, observation)]。
        task_plan (list): Agent 规划的任务列表。

    返回值：
        dict: 更新的对话步骤，包括思考、行动、观察和任务更新。
    """
    # 根据用户输入生成任务规划
    if not task_plan:
        task_plan = generate_task_plan(user_input)
    print("---------------------------------task_plan---------------------------------")
    print(task_plan)
    # 根据用户输入、历史记录和任务计划生成初步思考
    thought = reason_about_input(user_input, conversation_history, task_plan)
    print("---------------------------------thought---------------------------------")
    print(thought)
    # 根据思考决定行动
    action = decide_action(thought)

    # 使用 OpenAI 调用生成观察结果
    observation = execute_action_with_openai(action)

    # 根据观察结果更新任务计划
    updated_task_plan = update_task_plan(task_plan, observation)

    # 将当前步骤更新到对话历史中
    conversation_step = {
        "thought": thought,
        "action": action,
        "observation": observation,
        "task_plan": updated_task_plan
    }

    return conversation_step

def generate_task_plan(user_input):
    """根据用户输入生成初步任务规划。"""
    # plan_prompt = f"针对用户问题“{user_input}”，分解为多个可执行的子问题，确保问题能够逐步解决。"
    system_prompt = """你是一个擅长问题和任务推理的AI专家助手，能够将问题和任务分解成多个可执行的子问题，确保问题能够得到解决.
    要求：
    - 分解的子问题紧扣原始问题进行拆分，
    - 分解的子问题尽量简单，保持原子子问题
    - 仅返回解决原始问题的子问题列表
    # - 子问题与原始问题之间保持关联
    - 子问题是解决原始问题的一系列步骤和规划
    - 子问题是要通过sql查询获取答案，
    - 子问题拆解需要结合sql设计表的规范和逻辑，保持适度的拆分粒度
    - 子问题拆解尽量按照获取答案的顺序进行拆解，前一步推理的结果有可能作为下一步推理的输入，比如A->B, B->C, C为最终答案
    
    """
    system_prompt = """你是一个擅长问题和任务推理的AI专家助手，能够将问题和任务分解成多个可执行的子问题，确保问题能够得到解决.
    要求：
    - 分解的子问题紧扣原始问题进行拆分，每个子问题都有一个确定的答案
    - 仅返回解决原始问题的子问题列表
    - 子问题是要通过sql查询获取答案，
    - 子问题拆解需要结合sql设计表的规范和逻辑，保持适度的拆分粒度，不要输出sql语句
    - 子问题拆解尽量按照获取答案的顺序进行拆解，前一步推理的结果有可能作为下一步推理的输入，比如A->B, B->C, C为最终答案
    
    """
    response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",
                                              messages=[{"role": "system", "content": system_prompt},
                                                        {"role": "user", "content": user_input + "\n 步骤和规划为：\n"},],
                                              temperature=0.00001,
                                              # top_p=0.8,
                                              max_tokens=1024,
                                              extra_body={"repetition_penalty": 1.05,},
                                              )
    # print(response)
    return response.choices[0].message.content

def reason_about_input(user_input, conversation_history, task_plan):
    """根据输入、历史记录和当前任务推理下一步。"""
    if not conversation_history:
        return f"分析用户问题“{user_input}”，构建初步任务计划，并确定第一步行动。"
    last_observation = conversation_history[-1]['observation']
    return f"基于最新观察“{last_observation}”以及当前任务计划“{task_plan}”，推导解决用户问题“{user_input}”的下一步行动。"

def decide_action(thought):
    """根据思考决定采取的行动。"""
    if "分析" in thought:
        return "检索与用户问题相关的背景信息或数据。"
    elif "调整" in thought:
        return "总结已有信息并聚焦到更具体的子问题。"
    elif "澄清" in thought:
        return "向用户提出澄清性问题以获取更多上下文信息。"
    else:
        return "更新任务计划或验证当前进展的有效性。"

def execute_action_with_openai(action):
    """使用 OpenAI 接口执行行动并生成观察结果。"""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "你是一个智能助手，擅长分解任务并逐步解决问题。"},
                {"role": "user", "content": f"根据以下行动生成观察结果：\n行动: {action}\n观察:"}
            ],
            max_tokens=100,
            temperature=0.7
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"执行行动时出错: {e}"

def update_task_plan(task_plan, observation):
    """根据最新的观察结果更新任务计划。"""
    if "找到" in observation:
        task_plan.append("进一步分析检索到的相关数据。")
    elif "总结" in observation:
        task_plan.append("对总结的信息进行验证并形成初步结论。")
    elif "更新" in observation:
        task_plan.append("重新评估任务计划并执行下一个子问题。")
    return task_plan

# 多轮对话示例，带有任务规划和记忆功能
def main():
    print("欢迎使用增强版 ReAct Agent！请提出您的问题。")
    conversation_history = []
    task_plan = []

    while True:
        user_input = input("您：")
        if user_input.lower() in ["退出", "退出程序"]:
            print("再见！")
            break

        step = react_agent(user_input, conversation_history, task_plan)
        conversation_history.append(step)
        task_plan = step['task_plan']

        print(f"Agent 的思考：{step['thought']}")
        print(f"Agent 的行动：{step['action']}")
        print(f"Agent 的观察：{step['observation']}")
        print(f"当前任务计划：{step['task_plan']}")
        print("---")




In [ ]:

# 功率最大的发电机和是什么时候购买的
# 今年耗电量最大的设备是什么时候购买的

main()


欢迎使用增强版 ReAct Agent！请提出您的问题。


您： 今年耗电量最大的设备是什么时候购买的


---------------------------------task_plan---------------------------------
1. 获取今年所有设备的购买记录。
2. 获取今年所有设备的耗电量数据。
3. 计算每台设备的总耗电量。
4. 找出今年总耗电量最大的设备。
5. 获取该设备的购买时间。
---------------------------------thought---------------------------------
分析用户问题“今年耗电量最大的设备是什么时候购买的”，构建初步任务计划，并确定第一步行动。
Agent 的思考：分析用户问题“今年耗电量最大的设备是什么时候购买的”，构建初步任务计划，并确定第一步行动。
Agent 的行动：检索与用户问题相关的背景信息或数据。
Agent 的观察：执行行动时出错: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

当前任务计划：1. 获取今年所有设备的购买记录。
2. 获取今年所有设备的耗电量数据。
3. 计算每台设备的总耗电量。
4. 找出今年总耗电量最大的设备。
5. 获取该设备的购买时间。
---
